In [ ]:
# Importing all required packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import yfinance as yf
import datetime as dt
from pandas.tseries.offsets import MonthEnd
from dateutil.relativedelta import relativedelta
plt.style.use('fivethirtyeight')

# For sentiment analysis
from nltk.sentiment import SentimentIntensityAnalyzer as sia
from tqdm.notebook import tqdm

## SENTIMENT ANALYSIS OF STOCK TWEETS

In [66]:
import numpy as np 
import pandas as pd
from ast import Return
from textblob import TextBlob

# To read configs
import configparser

# To extarct the twitter data
import tweepy as tw
from tqdm import tqdm

# Datetime manipulations
import datetime as dt

# Import a needed packages
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# To be applied to the output to convert it to a probability range between 0-1
from scipy.special import softmax 

import re
import string

# Remove emojis
import demoji
 

# text processing libraries
import re
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from wordcloud import WordCloud

from tqdm import tqdm


# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dadai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

# authentication
auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth)

#### Extract tweets for specific user name

In [68]:
# Set parameters
limit =2500
start = dt.datetime(2019,9,14)
end = dt.datetime(2022,9,23)
data = []
columns_name = ['Date','User','Tweet Content']


# Define a function for tweets exactration
def extract_tweets(username):
    # Collect tweets, adding the needed parameters for your specific needed
    tweets = tw.Cursor(api.user_timeline,screen_name = username, count = 200, tweet_mode ='extended').items(limit)
    #Create a dataframe for tweets 
    global columns
    global data 
    global commentaries
    #Extract the tweets and other needed contents
    for comments in tweets:
        data.append([comments.created_at,comments.user.screen_name,comments.full_text])
        # Create the dataframe
        commentaries = pd.DataFrame(data, columns = columns_name)
        # Convert the tweets content to lower case for greater accessibility and context
        commentaries['Tweet Content'] = commentaries['Tweet Content'].str.casefold()
        # Show the dataframe 
        Return (commentaries)


# Create an empty df to stroe merged extracts
empty_df = pd.DataFrame(columns=columns_name)

In [69]:
# Extract for a list of users
user = ['GoldmanSachs','jpmorgan','BofA_Business','WellsFargo','Citi']

for values in user:
    extract_tweets(values)
    df1 = empty_df.append(commentaries)


# Convert it to Date
df1['Date'] = df1['Date'].dt.date
    


## _START FROM HERE_

In [3]:
# For consistency, the same tweet extracted as at the time of writing this was reimported directly.

df1 = pd.read_excel(r'Tweets.xlsx')
df1.head(3)

,Date,User,Tweet Content
0,2022-10-26,GoldmanSachs,our president and coo john waldron discusses t...
1,2022-10-25,GoldmanSachs,listen to brex co-ceo @hdubugras on scaling bu...
2,2022-10-24,GoldmanSachs,"following china’s national congress, our chief..."


In [71]:
# Check the least date captured and to which user
rslt = df1.loc[df1['Date'] == min(df1['Date'])]


# Create a table that aggregates the group by the date posted
df2 = (df1.groupby(['User'], as_index=False)
        .agg(**{'First Date':('Date', 'first'),
                'Last Date': ('Date', 'last')}))

# display
df2

,User,First Date,Last Date
0,BofA_Business,2022-10-26,2017-08-23
1,Citi,2022-10-26,2019-08-29
2,GoldmanSachs,2022-10-26,2020-05-21
3,WellsFargo,2022-10-27,2022-09-01
4,jpmorgan,2022-10-26,2018-02-20


### Keywords filter for the tweets

In [4]:
#To test for my 3rd challenge of finding tweets which contains certain words of interest
new_df = df1[df1['Tweet Content'].str.contains('algo|anti-city|anti-dilution|anti-speculation|anti-speculative|arbitrage|arbitrageur|asset management|asset-stripping|bagger|bear|bear market|bearish|blue-chip|bond|bourse|broker|bull|bull market|bullish|buy something in|buyback|callable|capital investment|carpetbagger|cash|cash out|cashout|city|copper-bottomed|crash|dead cat bounce|dealing|delist|dilution|disinvest|disinvestment|distributable|diversified|dividend|Dow Jones|electricals|equity|faller|Fibonacci |flotation |footsie |fund management |fund manager |future |gainer |gilt |go public idiom |growth-oriented |grubstake |haircut |head fake |hedging |holding |ICO |industrial |initial coin offering |insider dealing |intangible asset |investment |issue |list |listed company |mature |maturity |negotiable |non-directional|non-discretionary|non-distributable|non-rated |non-speculative |noncallable |nondiversified |OFEX |option |outgain |over-investment |par |payback period |plan |portfolio |post-crash |principal |public |pyramid scheme |pyramid selling |quant |redeemable |rentier |repurchase |rig| rig the market idiom |rogue trader |security |seed money |share |shareholder |small cap |sound |speculation |speculative |speculatively |spread betting |stag |stake |stakeholder |stock |stock exchange |stock market |stockbroker |stockbroker belt |stockbroking |stockholder |strip |subscribe |tender |the Big Board |the FTSE 100 |the grey market |the Nikkei index |the S&P 500 |S&P 500 |the Square Mile |trade |trader |trading |Treasury bond |unissued |unit trust |unlisted |venture capital |venture capitalist |Wall Street|administred prices | employment | inflation | aggregate demand | energy price | inflation | inflation rate | borrowing growth | energy prices |  inflationary pressure | broad money | equity prices | inflationary pressures')]

new_df

,Date,User,Tweet Content
4,2022-10-21,GoldmanSachs,rt @podcast_maze: new episode out now with pet...
5,2022-10-20,GoldmanSachs,how are top investors navigating today’s volat...
6,2022-10-19,GoldmanSachs,steve and connie ballmer's investment bridges ...
8,2022-10-18,GoldmanSachs,"rt @squawkcnbc: ""we've got a leading investmen..."
9,2022-10-18,GoldmanSachs,today we reported our 3q 2022 earnings results...
...,...,...,...
12477,2019-09-09,Citi,the financial agent mentor-protégé program bui...
12484,2019-09-05,Citi,"""citi’s $72 million investment in our new stat..."
12486,2019-09-05,Citi,rt @politicopress: can america become a leader...
12494,2019-09-04,Citi,what are some of the most pressing regulatory ...


In [5]:
# Create a copy to have a back up
new_dff = new_df.copy()

#Show Dataframe
new_dff

,Date,User,Tweet Content
4,2022-10-21,GoldmanSachs,rt @podcast_maze: new episode out now with pet...
5,2022-10-20,GoldmanSachs,how are top investors navigating today’s volat...
6,2022-10-19,GoldmanSachs,steve and connie ballmer's investment bridges ...
8,2022-10-18,GoldmanSachs,"rt @squawkcnbc: ""we've got a leading investmen..."
9,2022-10-18,GoldmanSachs,today we reported our 3q 2022 earnings results...
...,...,...,...
12477,2019-09-09,Citi,the financial agent mentor-protégé program bui...
12484,2019-09-05,Citi,"""citi’s $72 million investment in our new stat..."
12486,2019-09-05,Citi,rt @politicopress: can america become a leader...
12494,2019-09-04,Citi,what are some of the most pressing regulatory ...


In [94]:
'''
I am adding this next line to provide a solution for the date range differential in the final solution of the correlation between the sentiment and momentum
'''

# Convert Date to Datetime format.
new_dff['Date'] = pd.to_datetime(new_dff['Date'])



''' 
The user name can be switched for the 4 user names ('GoldmanSachs','jpmorgan','BofA_Business','Citi') to filter out different firms to be considered. #greater than the start date and smaller than the end date
'''
mask = (new_dff['Date'] > '2020-12-31') & (new_dff['Date'] <= '2021-11-30') & (new_dff['User'] == 'jpmorgan')

new_dff = new_dff.loc[mask]

# Returning the extract to new_df
new_df = new_dff.copy()

In [15]:
# View the dataframe
new_df.head(7)

,Date,User,Tweet Content
2793,2021-11-12,jpmorgan,agile payments infrastructures are redefining ...
2795,2021-11-12,jpmorgan,the city of lights is exhilarating during #par...
2802,2021-11-09,jpmorgan,an enchanting city that inspires and connects:...
2806,2021-11-04,jpmorgan,from machine learning to cybersecurity – techn...
2808,2021-11-03,jpmorgan,from one-on-one mentoring to hands-on experien...
2813,2021-10-25,jpmorgan,where innovation meets insights: j.p. morgan a...
2821,2021-10-15,jpmorgan,jennifer piepszak believes that taking the lon...


### DATA PREPROCESSING

In [16]:
def tweets_cleaner(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub(r"^@",'@user',text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def emoji_remover(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def text_processing(text):
    """
    - Tokenizes
    - Remove stopwords 
    - Clean text
    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    no_punctuation = tweets_cleaner(text)
    
    tokenized_text = tokenizer.tokenize(no_punctuation)
    remove_stopwords = [word for word in tokenized_text if word not in stopwords.words('english')]
    combined_text = ' '.join(remove_stopwords)
    
    return combined_text


#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [17]:
# Cleaning the tweet contents by replacing all unneeded character
new_df['Tweet Content'] = new_df['Tweet Content'].apply(lambda x: tweets_cleaner(x))

# Removing all emojis
new_df['Tweet Content'] = new_df['Tweet Content'].apply(lambda x: emoji_remover(x))

# Removing stop words
new_df['Tweet Content'] = new_df['Tweet Content'].apply(lambda x: text_processing(x))

In [18]:
# Defining the RoBERTA TimeLMS model importation
def import_roberta():
    """
    The function imports all the required model for using the pretrained roberta model
    """
    global tokenizer,roberta,model,labels

    # load the roberta model and tokenizer
    roberta = "cardiffnlp/twitter-roberta-base-sentiment"

    # Download the model from the website, USING .from_prtrained loads the model configuration file with the pretrained model weights
    model = AutoModelForSequenceClassification.from_pretrained(roberta,from_tf=False)

    # Load your tokenizer from the pretrained model
    tokenizer = AutoTokenizer.from_pretrained(roberta,from_tf=False)

    # create labels for your output as stated on the website
    labels = ['Negative','Neutral','Positive']


import_roberta()

In [19]:
# Extract sample to test the preprocessing
new_df2 = new_df.head(250)

# Creating empty lists for functions below 
result = []
opinion_output = []
opinion_probability = []

# Create a list to house individual  tweet response
Negative = []
Neutral = []
Positive = []

# Clear list
result.clear()
opinion_output.clear()
opinion_probability.clear()

In [59]:
# Define the sentiment classification process

def sentiment_model():
    """
    The function processes the sentiment and attaches the result as a new column - Sentiments
    """
    for tweet in new_df2['Tweet Content']:
        tweet_processed = tweet
        # Encode the processed tweets into a pytorch tensors to be passed into the roberta model 
        encoded_tweet = tokenizer(tweet_processed,return_tensors= 'pt')
        result.append(encoded_tweet)
        for i in result:
            # Pass the encoded tweet to the model to perform the sentiment analysis on the encode tweets
            sentiment_output = model(i['input_ids'],i['attention_mask'])
        opinion_output.append(sentiment_output)
        #Iterate and extract logit result
        for j in opinion_output:
            mscores = j[0][0].detach().numpy()
            # Convert mscores to probabilities
            mscores = softmax(mscores)
            # Append the list of sentiments converted as probability into a list
            opinion_probability.append(mscores)
            # Slice the list to detach various opinion into new columns
            for k in opinion_probability:
                Neg = k[0]
                Neut = k[1]
                Pos = k[2]
        Negative.append(Neg)
        Neutral.append(Neut)
        Positive.append(Pos)

    #return result,opinion_output,mscores

sentiment_model()

In [60]:
# Input the results into the dataframe 
new_df2['Negative'] = np.array(Negative)
new_df2['Neutral'] = np.array(Neutral)
new_df2['Positive'] = np.array(Positive)

In [61]:
# The new column houses the maximum most likely sentiment
new_df2['Most Likely']= new_df2[["Negative", "Neutral","Positive"]].max(axis=1)

new_df2

,Date,User,Tweet Content,Negative,Neutral,Positive,Most Likely
2793,2021-11-12,jpmorgan,agile payments infrastructures redefining futu...,0.025943,0.811521,0.162536,0.811521
2795,2021-11-12,jpmorgan,city lights exhilarating honored partners cons...,0.001168,0.037968,0.960864,0.960864
2802,2021-11-09,jpmorgan,enchanting city inspires connects art enthusia...,0.001921,0.247319,0.750760,0.750760
2806,2021-11-04,jpmorgan,machine learning cybersecurity technology shap...,0.008927,0.752064,0.239009,0.752064
2808,2021-11-03,jpmorgan,oneonone mentoring handson experience new prog...,0.008096,0.425706,0.566199,0.566199
...,...,...,...,...,...,...,...
3111,2021-02-10,jpmorgan,rt cityam exlabour mp chuka umunna hired jpmor...,0.174036,0.800406,0.025559,0.800406
3113,2021-02-08,jpmorgan,three leading executives named diversitywomans...,0.011116,0.797792,0.191092,0.797792
3116,2021-02-03,jpmorgan,factors accelerating esg investment flows jp m...,0.027222,0.899902,0.072876,0.899902
3121,2021-01-27,jpmorgan,proud recognized greenwich associates global l...,0.001324,0.064803,0.933873,0.933873


In [6]:
'''
Export the sentiment results to csv file for all users and save for further data manipulation and also to avoid running the model multiple times
'''
# new_df2.to_csv('BofA_Business.csv', index=False) (change the file name as new models are exported)

'''
Since I have processed all the above users, I can import the result back without rerunning the model
'''
# Import results AFTER PERFOMING DATA MANIPULATION IN EXCEL
new_df2 = pd.read_csv(r'Jpmorganresult.csv')

new_df2.head(5)

,Date,User,Tweet Content,Negative,Neutral,Positive,Most Likely,Maximum Sentiment
0,12/11/2021,jpmorgan,agile payments infrastructures redefining futu...,0.025943,0.811521,0.162536,0.811521,Neutral
1,12/11/2021,jpmorgan,city lights exhilarating honored partners cons...,0.001168,0.037968,0.960864,0.960864,Positive
2,09/11/2021,jpmorgan,enchanting city inspires connects art enthusia...,0.001921,0.247319,0.750760,0.750760,Positive
3,04/11/2021,jpmorgan,machine learning cybersecurity technology shap...,0.008927,0.752064,0.239009,0.752064,Neutral
4,03/11/2021,jpmorgan,oneonone mentoring handson experience new prog...,0.008096,0.425706,0.566199,0.566199,Positive
